# Exploratory part of Intel Sensors data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#df = pd.read_csv("data.csv")
# Read the file using Pandas, specifying space (' ') as the separator
df = pd.read_csv("data.txt", sep=' ', header=None)

# You can assign column names based on the dataset's description
df.columns = ['date', 'time', 'epoch', 'moteid', 'temperature', 'humidity', 'light', 'voltage']
df.sort_values(by=['epoch', 'moteid'], inplace=True)

print(df.describe())
df.to_csv("data.csv", index=False)

              epoch        moteid   temperature      humidity         light  \
count  2.313682e+06  2.313156e+06  2.312781e+06  2.312780e+06  2.219804e+06   
mean   3.303993e+04  2.854412e+01  3.920700e+01  3.390814e+01  4.072110e+02   
std    1.836852e+04  5.062408e+01  3.741923e+01  1.732152e+01  5.394276e+02   
min    0.000000e+00  1.000000e+00 -3.840000e+01 -8.983130e+03  0.000000e+00   
25%    1.757200e+04  1.700000e+01  2.040980e+01  3.187760e+01  3.956000e+01   
50%    3.332700e+04  2.900000e+01  2.243840e+01  3.928030e+01  1.582400e+02   
75%    4.778900e+04  4.100000e+01  2.702480e+01  4.358550e+01  5.372800e+02   
max    6.553500e+04  6.540700e+04  3.855680e+02  1.375120e+02  1.847360e+03   

            voltage  
count  2.313156e+06  
mean   2.492552e+00  
std    1.795743e-01  
min    9.100830e-03  
25%    2.385220e+00  
50%    2.527320e+00  
75%    2.627960e+00  
max    1.856000e+01  


In [2]:
print(df[['temperature', 'humidity', 'light', 'voltage']].corr())

             temperature  humidity     light   voltage
temperature     1.000000 -0.708732  0.021822 -0.758731
humidity       -0.708732  1.000000 -0.091569  0.512332
light           0.021822 -0.091569  1.000000  0.064854
voltage        -0.758731  0.512332  0.064854  1.000000


In [3]:
# Example: identify temperature outliers
outliers = df[df['temperature'] > df['temperature'].quantile(0.99)]
outliers

date             time  epoch   moteid  temperature    humidity  \
2313155  2004-03-05  08:29:04.505813  18182  65407.0      294.251         NaN   
2313153  2004-03-23  21:32:16.223693  35112   6485.0      262.656         NaN   
1832305  2004-03-19   13:22:51.76293  59089     43.0      385.568    0.580611   
1662602  2004-03-21  13:50:07.929237  64903     39.0      175.681 -455.449000   

            light   voltage  
2313155       NaN  0.013056  
2313153       NaN  0.393324  
1832305  1.809920  2.454210  
1662602  0.562671  0.027109

In [4]:
# check missing values
print(df.isnull().sum(), "\n")
# check types
print(df.dtypes, "\n")
# unique values counts 
print(df['moteid'].nunique(), "\n")

date               0
time               0
epoch              0
moteid           526
temperature      901
humidity         902
light          93878
voltage          526
dtype: int64 

date            object
time            object
epoch            int64
moteid         float64
temperature    float64
humidity       float64
light          float64
voltage        float64
dtype: object 

61 



# Streaming example with Intel Sensors data

In [ ]:
import time
import pandas as pd

def load_sensor_coordinates(coordinate_file):
    # Load the coordinates into a DataFrame
    return pd.read_csv(coordinate_file)

def simulate_stream(data_file, coordinate_df, delay=1):
    for chunk in pd.read_csv(data_file, chunksize=1):
        # Join chunk with coordinate data based on moteid
        chunk_with_coords = chunk.merge(coordinate_df, on='moteid', how='left')
        process_data(chunk_with_coords)
        time.sleep(delay)

def process_data(data):
    # Example processing function
    print(data)

# Paths to your files
data_file_path = 'data.csv'
coordinate_file_path = 'coordinates.csv'

# Load coordinates
coordinates = load_sensor_coordinates(coordinate_file_path)

# Start the simulation
simulate_stream(data_file_path, coordinates)


# Streaming example with News API

In [ ]:
import requests
import time

# Replace 'YOUR_API_KEY' with your actual NewsAPI key
api_key = 'YOUR_API_KEY'
url = 'https://newsapi.org/v2/top-headlines'

# Define parameters: country, category, etc. as per your requirement
params = {
    'apiKey': api_key,
    'country': 'us',  # Example: Fetch news from the United States
}

def fetch_news():
    response = requests.get(url, params=params)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        for article in articles:
            print(f"Title: {article['title']}")
            print(f"Description: {article['description']}\n")
    else:
        print("Failed to fetch news")

# Fetch news at regular intervals (e.g., every 3600 seconds/1 hour)
interval = 3600  # In seconds
while True:
    fetch_news()
    time.sleep(interval)


# Streaming example with OpenWeatherMap API

In [ ]:
import requests
import time

# Replace 'YOUR_API_KEY' with your actual OpenWeatherMap API key
api_key = 'YOUR_API_KEY'
url = 'http://api.openweathermap.org/data/2.5/weather'

# Define parameters: city, units, etc.
params = {
    'q': 'London',  # Example: London, UK
    'appid': api_key,
    'units': 'metric'  # or 'imperial' for Fahrenheit
}

def fetch_weather():
    response = requests.get(url, params=params)
    if response.status_code == 200:
        weather = response.json()
        print(f"Weather in {weather['name']}: {weather['weather'][0]['description']}")
        print(f"Temperature: {weather['main']['temp']}°C\n")
    else:
        print("Failed to fetch weather data")

# Fetch weather at regular intervals (e.g., every 3600 seconds/1 hour)
interval = 3600  # In seconds

while True:
    fetch_weather()
    time.sleep(interval)
